In [ ]:
import httpx
import pandas as pd

response = httpx.post('https://m.suanya.com/restapi/soa2/14666/json/GetBookingByStationV3ForPC',
                      headers={
                          "authority": "m.suanya.com",
                          "accept": "*/*",
                          "accept-language": "zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6",
                          "cache-control": "max-age=0",
                          "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
                          "if-modified-since": "Thu, 01 Jan 1970 00:00:00 GMT",
                          "origin": "https://www.suanya.com",
                          "referer": "https://www.suanya.com/",
                          "sec-ch-ua": '"Not A(Brand";v="99", "Microsoft Edge";v="121", "Chromium";v="121"',
                          "sec-ch-ua-mobile": "?0",
                          "sec-ch-ua-platform": '"Windows"',
                          "sec-fetch-dest": "empty",
                          "sec-fetch-mode": "cors",
                          "sec-fetch-site": "same-site"
                      },
                      data={
                          "ArriveStation": "上海",
                          "ChannelName": "ctrip.pc",
                          "DepartDate": "2024-02-19",
                          "DepartStation": "赣榆"
                      }
                      )


In [14]:
# 使用orgjson格式化输出 
import orjson

text = response.text
data = orjson.loads(text)
TrainItems = data["ResponseBody"]["TrainItems"]

In [15]:
# 使用列表推导式和字典推导式提取所需信息
data = [
    {
        "车次": item["TrainName"],
        "开始时间": item["StartTime"],
        "结束时间": item["EndTime"],
        "耗时": f'{item["UseTime"] // 60}.{item["UseTime"] % 60}',
        "一等座价格": next(
            (ticket["Price"] for ticket in item["TicketResult"]["TicketItems"] if ticket["SeatTypeName"] == "一等座"),
            None),
        "一等座余票量": next((ticket["Inventory"] for ticket in item["TicketResult"]["TicketItems"] if
                              ticket["SeatTypeName"] == "一等座"), None),
        "二等座价格": next(
            (ticket["Price"] for ticket in item["TicketResult"]["TicketItems"] if ticket["SeatTypeName"] == "二等座"),
            None),
        "二等座余票量": next((ticket["Inventory"] for ticket in item["TicketResult"]["TicketItems"] if
                              ticket["SeatTypeName"] == "二等座"), None),
    }
    for item in TrainItems
]

# 创建DataFrame
df = pd.DataFrame(data)
df

,车次,开始时间,结束时间,耗时,一等座价格,一等座余票量,二等座价格,二等座余票量
0,D2131,08:41,12:41,4.0,352,0,221,0
1,D2923,10:11,14:41,4.30,372,0,232,0
2,D2135,10:49,15:10,4.21,352,0,220,0
3,G7585,13:03,17:12,4.9,463,0,290,0
4,D2145,15:50,19:09,3.19,371,0,233,0
5,D2927,18:22,22:38,4.16,399,0,250,0
6,D2151,18:34,22:32,3.58,371,0,233,0
7,D4117,19:41,00:23,4.42,371,99,233,99
